In [1]:
import yfinance as yf
import talib
from stockstats import StockDataFrame as Sdf
import pickle
import os
import requests
import pandas as pd

In [2]:
from fredapi import Fred
fred = Fred(api_key='X')

Bitcoin  
Ethereum   
Litecoin  
XRP  
Monero  

OHLCV Variables

In [ ]:
def fetch_ohlcv_data(symbol, comparison_symbol, all_data=True, limit=2000, aggregate=1):
    """ Fetch historical OHLCV data for a given pair of symbols. """
    url = "https://min-api.cryptocompare.com/data/v2/histoday"
    api_key = 'your_api_key_here'  # Replace with your CryptoCompare API key
    params = {
        'fsym': symbol,
        'tsym': comparison_symbol,
        'limit': limit,
        'aggregate': aggregate,
        'api_key': api_key
    }
    if all_data:
        params['allData'] = 'true'
    
    response = requests.get(url, params=params)
    data = response.json()['Data']['Data']
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    
    return df

In [3]:
INDEX = ["BTC-USD", "LTC-USD", "BTC-USD", "XRP-USD", "BTC-USD"]
start_date = "2017-11-30"
end_date = "2024-03-31"

data = {}

#['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
for index in INDEX:
    data[index] = yf.download(index, start=start_date, end=end_date, progress=False)

INDICATORS

*Note on last three added
ATR (Average True Range): This indicator will add a measure of market volatility that can be very useful for risk management, particularly in adjusting stop-loss orders based on how much the stock price typically fluctuates.
ADX (Average Directional Movement Index): Including ADX can provide a clearer picture of the trend strength. It helps distinguish between trending and non-trending conditions, complementing the MACD and RSI you already have.
Ichimoku Cloud (Ichimoku): This is a comprehensive indicator that provides information on support and resistance levels, trend direction, momentum, and provides signals for buying and selling. It’s complex but offers a broad view of the market at a glance.

In [4]:
INDICATORS = [ #Same indicators as the ensamble
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",
    "close_60_sma",
    'mfi', #ADDED
    'atr_14',
    'adx',
    'ichimoku',
    'supertrend_ub',
    'supertrend_lb'

]
#"MFI"
for index in INDEX:
    for indicator in INDICATORS:
            ind = Sdf.retype(data[index].copy())
            data[index][indicator] = ind[indicator]

CNNPRED

In [5]:
for index, df in data.items():
    # Calculate Momentum (MOM) for 1, 2, and 3 days before
    df['mom'] = df['Close'].pct_change(periods=1)  #NOT INCLUDED IN TABLE!!!**********
    df['mom1'] = df['mom'].shift(1)  # Shifted 1 day to get the return of 2 days ago
    df['mom2'] = df['mom'].shift(2)  # Shifted 2 days to get the return of 3 days ago
    df['mom3'] = df['mom'].shift(3)  # Shifted 3 days to get the return of 4 days ago
    #df.drop('mom', axis=1)

    # Calculate Rate of Change (ROC) for 5, 10, 15, and 20 days
    df['ROC_5'] = talib.ROC(df['Close'], timeperiod=5)
    df['ROC_10'] = talib.ROC(df['Close'], timeperiod=10)
    df['ROC_15'] = talib.ROC(df['Close'], timeperiod=15)
    df['ROC_20'] = talib.ROC(df['Close'], timeperiod=20)

    # Calculate Exponential Moving Average (EMA) for 10, 20, 50, and 200 days
    df['EMA_10'] = talib.EMA(df['Close'], timeperiod=10)
    df['EMA_20'] = talib.EMA(df['Close'], timeperiod=20)
    df['EMA_50'] = talib.EMA(df['Close'], timeperiod=50)
    df['EMA_200'] = talib.EMA(df['Close'], timeperiod=200)

Tech Stocks  
Marathon Digital Holdings Inc. - MARA  
Riot Platforms Inc. - RIOT  
MicroStrategy Inc. - MSTR  
Coinbase Global Inc. - COIN ---- REMOVED  
Galaxy Digital Holdings Ltd - GLXY ---- REMOVED  
Block Inc. - SQ  
NVIDIA Corp. - NVDA  
Advanced Micro Devices Inc. - AMD  
PayPal Holdings Inc. - PYPL  
CME Group Inc. - CME  
  
Added:  
IBM Corporation (IBM)
Intercontinental Exchange (ICE)

MFI (Money Flow Index): As you've chosen, this will provide insight into both price changes and trading volume, helping identify overbought or oversold conditions with a volume-weighted approach.
MACD (Moving Average Convergence Divergence): This remains a solid choice for assessing the direction and momentum of trends. MACD is helpful for spotting both trend reversals and momentum strength without incorporating volume, thus complementing the MFI's sensitivity to volume changes.
Bollinger Bands: This indicator will add an understanding of market volatility and price levels relative to previous performance. Bollinger Bands measure market volatility by plotting standard deviations above and below a simple moving average of the price. When the bands widen, it indicates increased volatility, and when they contract, it suggests decreased volatility.

In [6]:
STOCKS = [ 'MARA', 'RIOT', 'MSTR', 'SQ', 'NVDA', 'AMD', 'PYPL', 'CME', 'IBM', 'ICE']
INDICATORS = [
    "macd",
    "mfi",
    "boll",
]

for index in INDEX:
    for stock in STOCKS:
        stock_data = yf.download(stock, start=start_date, end=end_date, progress=False)
        for indicator in INDICATORS:
                ind = Sdf.retype(stock_data.copy())
                data[index][indicator + "-" + stock] = ind[indicator]

In [7]:
EXCHANGE_RATES = [
    "EURUSD=X",  # Euro to US Dollar
    "USDJPY=X",  # US Dollar to Japanese Yen
    "GBPUSD=X",  # British Pound to US Dollar
    "AUDUSD=X",  # Australian Dollar to US Dollar
    "USDCAD=X",  # US Dollar to Canadian Dollar
    "USDCHF=X",  # US Dollar to Swiss Franc
    "USDSEK=X",  # US Dollar to Swedish Krona
    "USDSGD=X",  # US Dollar to Singapore Dollar
    "USDHKD=X",  # US Dollar to Hong Kong Dollar
    "USDINR=X"   # US Dollar to Indian Rupee
]

INDICATORS = "macd"

for index in INDEX:
    for exchange_rate in EXCHANGE_RATES:
        data[index][indicator + "-" + exchange_rate] = yf.download(exchange_rate, start=start_date, end=end_date, progress=False)['Close']
        data[index][indicator + "-" + exchange_rate + "-" + INDICATORS] = Sdf.retype(stock_data.copy())[INDICATORS]

In [8]:
datasetName = "cryptoset"
datasetDir = "./Datasets"

os.makedirs(datasetDir, exist_ok=True)
datasetPath = os.path.join(datasetDir, datasetName) + ".pkl"


with open(datasetPath, 'wb') as file:
    pickle.dump(data, file)